In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
print('tensorflow {}'.format(tf.__version__))
print("keras {}".format(keras.__version__))

tensorflow 2.9.2
keras 2.9.0


In [ ]:
def background_delete_save(_img,cnt):
  im = Image.open(_img)
  img = np.array(im)
  height, width, channel = img.shape
  bgr = img.astype(np.float)/255.0
  b, g, r = cv.split(bgr)
  h = np.zeros((height,width), dtype=np.float)
  s = np.zeros((height,width),dtype=np.float)
  v = np.max(bgr, axis=2)
  for i in range(height):
    for j in range(width):
        if v[i][j] == 0:
            h[i][j] = 0
            s[i][j] = 0
        else:
            min_rgb = min(bgr[i][j])
            
            s[i][j] = 1 - (min_rgb / v[i][j])
            
            if v[i][j] == r[i][j]:
                h[i][j] = 60 * (g[i][j] - b[i][j]) / (v[i][j] - min_rgb + 0.00001)
            elif v[i][j] == g[i][j]:
                h[i][j] = 120 + (60 * (b[i][j] - r[i][j])) / (v[i][j] - min_rgb + 0.00001)
            elif v[i][j] == b[i][j]:
                h[i][j] = 240 + (60 * (r[i][j] - g[i][j])) / (v[i][j] - min_rgb + 0.00001)
            if h[i][j] < 0:
                h[i][j] += 360
            h[i][j] /= 360
  #metric = (h>h[0][0]+0.03) | (h<h[0][0]-0.03)
  metric = (h>h[0][0]+0.015) | (h<h[0][0]-0.015) | (s>s[0][0]+0.045) | (s<s[0][0]-0.045)
  metric = metric.reshape(height,width,1)
  new_img = Image.fromarray(img*metric)
  new_img.save('/content/drive/MyDrive/8000_pills/{}.jpg'.format(cnt))


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cnt = 0
for i in range(8144):
  background_delete_save('/content/drive/MyDrive/pills/{}.jpg'.format(i),cnt)
  cnt += 1
  if i%81==0:
    print(i//81,'% Done \n')